In [3]:
import pandas as pd
import spacy

In [21]:
nlp1 = spacy.load("de_core_news_sm")  # Model 1
nlp2 = spacy.load('de_core_news_md')  # Model 2
nlp3 = spacy.load('de_core_news_lg')  # Model 3

In [8]:
df_german = pd.read_csv('../scraping/data/extractor_all_articles_20minuten.csv')
df_french = pd.read_csv('../scraping/data/extractor_all_articles_20minutes.csv')
df_italian = pd.read_csv('../scraping/data/extractor_all_articles_20minuti.csv')

In [19]:
def extract_locations_with_voting_system(text):
    # Process the text with all three models
    doc1 = nlp1(text)
    doc2 = nlp2(text)
    doc3 = nlp3(text)
    
    # Extract locations from each model
    locations1 = {ent.text for ent in doc1.ents if ent.label_ == 'LOC'}
    locations2 = {ent.text for ent in doc2.ents if ent.label_ == 'LOC'}
    locations3 = {ent.text for ent in doc3.ents if ent.label_ == 'LOC'}
    
    # Perform a majority vote
    all_locations = locations1 | locations2 | locations3
    final_locations = {loc for loc in all_locations if sum([loc in locations for locations in [locations1, locations2, locations3]]) >= 2}
    
    return final_locations

In [ ]:
def extract_locations(text):
    doc = nlp1(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
    return locations

In [23]:
df_german['Voted_Locations'] = df_german['Content'].apply(extract_locations_with_voting_system)

In [24]:
df_german_exploded = df_german.explode('Voted_Locations')

In [25]:
location_counts_voted = df_german_exploded['Voted_Locations'].value_counts()

In [26]:
location_counts_voted

Schweiz            131
der Schweiz         83
Zürich              70
Franken             61
Deutschland         58
                  ... 
Schöneichtunnel      1
Schwamendingen       1
Deftiges             1
Süsses               1
Luchswiesen          1
Name: Voted_Locations, Length: 2515, dtype: int64

In [29]:
# Apply the function to the 'Content' column
df_german['Locations'] = df_german['Content'].apply(extract_locations)

In [30]:
# Count occurrences of each location
location_counts = df_german['Locations'].explode().value_counts()

In [31]:
# Display the results
print(location_counts)

Schweiz                        235
Israel                         130
Franken                        116
Zürich                         106
Deutschland                     93
                              ... 
Leverkusen-Trainer               1
Bayern-Coach                     1
Guardiola                        1
Dietlikon                        1
VBZ-Haltestelle Luchswiesen      1
Name: Locations, Length: 3442, dtype: int64
